1\. Write a function that converts number representation, bin<->dec<->hex. (Clearly using the corresponding python built-in functions is not fair..)

In [20]:
def number_conversion(n, from_number, to_number):

    # hexadecimal digits
    digits = "0123456789ABCDEF"

    match (from_number, to_number):
        # decimal to binary
        case ("dec", "bin"):
            bin = ""

            if n==0: return "0"
            while n>0:
                bin = str(n%2)+bin
                n //= 2
            return bin
        
        # decimal to hexadecimal
        case ("dec", "hex"):
            hex = ""

            if n==0: return "0"
            while n>0:
                hex = digits[n%16]+hex
                n //= 16
            return hex
        
        # binary to decimal
        case ("bin", "dec"):
            n = str(n)
            dec, pow = 0, 0
        
            for i in reversed(n):
                dec += int(i)*(2**pow)
                pow += 1
            return dec
        
        # hexadecimal to decimal
        case ("hex", "dec"):
            n = str(n)
            dec, pow = 0, 0
            
            for i in reversed(n):
                dec += digits.index(i)*(16**pow)
                pow += 1
            return dec

# Examples, I use 123 as number
dec_number = 123
bin_number = "1111011"
hex_number = "7B"

print("Decimal number:")
print(number_conversion(dec_number, "dec", "bin"))
print(number_conversion(dec_number, "dec", "hex"))

print("\nBinary number:")
print(number_conversion(bin_number, "bin", "dec"))

print("\nHexadecimal number:")
print(number_conversion(hex_number, "hex", "dec"))

Decimal number:
1111011
7B

Binary number:
123

Hexadecimal number:
123


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [22]:
def float_decode(word):
    #bit 31     -> sign
    #bit 30-23  -> exponent
    #bit 22-0   -> mantissa

    sign = 0x40000000
    exp = 0x7F800000
    mantissa = 0x7FFFFF
    masks = [2**i for i in range(23)]
    bias = 127

    sign = (word & sign) >> 30
    exponent = (word & exp) >> 23
    mantissa = word & mantissa

    number = 1
    for i, mask in enumerate(masks):
        number+= ((mantissa&mask) >> i)*(2**(-len(masks)+i))
    number*=(-1)**sign*2**(exponent-bias) 

    return(number)

print(float_decode(0xc0900000))

-4.5


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [29]:
def overflow_limit(value):

    while True:
        prev_value = value
        value *= 2

        if value == float('inf'): 
            return prev_value
        elif value == 0:  
            return "Unexpected underflow during overflow test."

def underflow_limit(value):

    while value > 0:
        prev_value = value
        value /= 2
        if value == 0: 
            return prev_value


print(f"Overflow limit: {overflow_limit(1.0)}")
print(f"Underflow limit: {underflow_limit(1.0)}")

Overflow limit: 8.98846567431158e+307
Underflow limit: 5e-324


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [36]:
def machine_precision(n1, n2):
    i = 1
    while (n1 != n2):
        n2 = n1+10**(-i)
        i += 1
    
    return i-1

print(f"Machine precision: {machine_precision(1, 2)}")

Machine precision: 16


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [44]:
from math import sqrt

def quadratic_sol(a, b, c):
    x1 = (-b + sqrt(b**2 - 4*a*c)) / (2*a)
    x2 = (-b - sqrt(b**2 - 4*a*c)) / (2*a)

    return x1, x2

def quadratic_sol_v2(a, b, c):
    x1 = (2*c)/(-b+sqrt(b**2-4*a*c))
    x2 = (2*c)/(-b-sqrt(b**2-4*a*c))

    return x1, x2

def quadratic_sol_acc(a, b, c):
    if b > 0:
        x1 = (-b - sqrt(b**2-4*a*c)) / (2*a)
        x2 = (2*c) / (-b - sqrt(b**2-4*a*c))
    else:
        x1 = 2*c/(-b + sqrt(b**2 - 4*a*c))
        x2 = (-b + sqrt(b**2 - 4*a*c)/(2*a))
    
    return x1, x2



a = 0.001
b = 1000
c = 0.001

print(quadratic_sol(a, b, c))
print(quadratic_sol_v2(a, b, c))
print(quadratic_sol_acc(a, b, c))

(-9.999894245993346e-07, -999999.999999)
(-1000010.5755125057, -1.000000000001e-06)
(-999999.999999, -1.000000000001e-06)


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [47]:
# A
# Define the function
def f(x):
    return x*(x-1)

# Define the derivative
def der_f(x):
    return 2*x - 1

# Define variables
delta = 10**(-2)
x = 1

# Compute derivative both with the limit and the definition
df_dx = (f(x+delta) - f(x)) / delta
print(df_dx)
print(der_f(x))

# B
deltas = [10**-4, 10**-6, 10**-8, 10**-10, 10**-12, 10**-14]
for i in deltas:
    print((f(x+i) - f(x)) / i)

1.010000000000001
1
1.0000999999998899
1.0000009999177333
1.0000000039225287
1.000000082840371
1.0000889005833413
0.9992007221626509


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [62]:
# A
from math import sqrt, pi

# Define the function
def f(x):
    return sqrt(1-x**2)

# Define the integral
def integral(domain, h):
    I = 0
    for i in domain:
        I += h*f(i)
    
    return I

N = 100
h = 2/N
domain = [i*h -1 for i in range(N+1)]

print("Integral value: ", integral(domain, h))
print("Difference from original value: ", pi/2 - integral(domain, h))

Integral value:  1.5691342555492505
Difference from original value:  0.0016620712456461018
